### Initiialise script

In [10]:
import numpy as np
import os
import glob
import pandas as pd 
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

def make_antecedent_conditions (results, timeperiod, statistic, days):
    # Method to get list of dates (is not working with ReFH2)
    # dates = []
    # for day in range(days,0,-1):
    #   dates.append(start_of_event - (oneday*day))
    
    # Get a list of dates (for some reason when create within Python, ReFH2 doesn't accept them)
    # Use an existing csv which has dates which work
    toedit = pd.read_csv("LinDykeAntecedentConditions/wholeyear/FormattedDates.csv", header=None)
    times = toedit[len(toedit)-days:][0].values
    
    # Create dataframe with results
    df = pd.DataFrame({"Times":times, 'Rainfall':results[statistic]})
    # Save to csv    
    df.to_csv("LinDykeAntecedentConditions/{}/lindyke_daily_antecedent_conditions_{}_{}days.csv".format(timeperiod, statistic, days),
                    header = False,index= False, date_format='%Y/%m/%d %H:%M')
    return df

In [11]:
catchment_name = 'WykeBeck' #WykeBeck' 'LinDyke'

### Load the times related to the hourly data for each catchment grid cell 

In [15]:
times = np.load('Rainfall/{}/wholeyear/times.npy'.format(catchment_name, timeperiod), allow_pickle=True)

### Get a list of all the individual grid cell .npy files
(Don't include files containing times and the total)

In [16]:
npy_files = glob.glob('Rainfall/{}/wholeyear/*.npy'.format(catchment_name))#[:-2]
npy_files = [ x for x in npy_files if "times" not in x and "all_the_data" not in x]

### Create an array containing all the daily values for the grid cells in the catchment in one place

In [17]:
# Create an array to store the daily values for all the grid cells in the catchment
all_daily_sums_jja = np. array([])
all_daily_sums_wholeyear = np.array([])

# Add the daily data for each grid cell
for file in npy_files:
    one_cell = np.load(file, allow_pickle = True)
    # Convert to a dataframe with the dates attached 
    df = pd.DataFrame({'Dates': pd.to_datetime(times), 'Precipitation': one_cell})
    # Resample from hours to days
    daily_sums = df.set_index('Dates').resample('D')['Precipitation'].sum()
    # Resampling process adds back in the days which are in the months we wanted to exclude
    daily_sums_df =pd.DataFrame(daily_sums)
    daily_sums_df.reset_index(inplace=True)
    # Daily sums - JJA
    daily_sums_df_jja= daily_sums_df[daily_sums_df['Dates'].dt.month.isin([6,7,8])]
    
    # Join to array containing data from across all the cells
    all_daily_sums_wholeyear = np.append(all_daily_sums_wholeyear, daily_sums_df['Precipitation'])
    all_daily_sums_jja = np.append(all_daily_sums_jja, daily_sums_df_jja['Precipitation']) 

### Find the mean and Xth percentile daily precipitation values across the area covering the catchment 

### JJA

In [32]:
stats = ["25th Percentile", '50th Percentile','75th Percentile', '80th Percentile', '81st Percentile',
         '88th Percentile', '90th Percentile','95th Percentile', '99th Percentile', 'mean']

results_jja = {}
for stat in stats.copy():
    if stat == 'mean':
        results_jja[str(stat)] = np.mean(all_daily_sums_jja)
    else:
        results_jja[str(stat)] = np.percentile(all_daily_sums_jja, int(stat[0:2]))
results_jja    

{'25th Percentile': 0.0,
 '50th Percentile': 0.10000000149011612,
 '75th Percentile': 1.8000000715255737,
 '80th Percentile': 2.799999952316284,
 '81st Percentile': 3.0,
 '88th Percentile': 5.099999904632568,
 '90th Percentile': 6.099999904632568,
 '95th Percentile': 10.0,
 '99th Percentile': 24.799999237060547,
 'mean': 2.0224730483805358}

### Whole year

In [33]:
stats = ["25th Percentile", '50th Percentile','75th Percentile', '80th Percentile', '81st Percentile',
         '88th Percentile', '90th Percentile','95th Percentile', '99th Percentile', 'mean']

results_wholeyear = {}
for stat in stats.copy():
    if stat == 'mean':
        results_wholeyear[str(stat)] = np.mean(all_daily_sums_wholeyear)
    else:
        results_wholeyear[str(stat)] = np.percentile(all_daily_sums_wholeyear, int(stat[0:2]))
results_wholeyear    

{'25th Percentile': 0.0,
 '50th Percentile': 0.20000000298023224,
 '75th Percentile': 2.0,
 '80th Percentile': 2.9000000953674316,
 '81st Percentile': 3.1000001430511475,
 '88th Percentile': 5.099999904632568,
 '90th Percentile': 5.900000095367432,
 '95th Percentile': 9.399999618530273,
 '99th Percentile': 19.5,
 'mean': 1.9115540543532805}

### Save results to file

In [31]:
def make_antecedent_conditions (results, timeperiod, statistic, days):
    # Method to get list of dates (is not working with ReFH2)
    # dates = []
    # for day in range(days,0,-1):
    #   dates.append(start_of_event - (oneday*day))
    
    # Get a list of dates (for some reason when create within Python, ReFH2 doesn't accept them)
    # Use an existing csv which has dates which work
    toedit = pd.read_csv("AntecedentConditions/LinDyke/wholeyear/FormattedDates.csv", header=None)
    times = toedit[len(toedit)-days:][0].values
    
    # Create dataframe with results
    df = pd.DataFrame({"Times":times, 'Rainfall':results[statistic]})
    # Save to csv    
    df.to_csv("AntecedentConditions/{}/{}/daily_antecedent_conditions_{}_{}days.csv".format(catchment_name, timeperiod,statistic, days),
                    header = False,index= False, date_format='%Y/%m/%d %H:%M')
    return df

In [35]:
# Define when the event will start
start_of_event = datetime(2022, 8, 1, 0, 0)
oneday = timedelta(days=1)
for statistic in stats:
    for days in [4,365,15]:
        rainfall = make_antecedent_conditions(results_jja, 'jja',  statistic,days)
        rainfall = make_antecedent_conditions(results_wholeyear, 'wholeyear',  statistic,days)